In [43]:
#Downloads the berkley dataset to be cleaned.
import datasets
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary') 
df = dataset['train'].to_pandas()

#Convert the hate_speech_score into a true or a false
for i in range(len(df)):
    if df.loc[i,"hate_speech_score"] <= 0:
        df.loc[i,"classification"] = '__label__Acceptable'
    else:
        df.loc[i,"classification"] = '__label__Unacceptable'

Using custom data configuration ucberkeley-dlab--measuring-hate-speech-f91f636a830ad73c
Reusing dataset parquet (C:\Users\patri\.cache\huggingface\datasets\parquet\ucberkeley-dlab--measuring-hate-speech-f91f636a830ad73c\0.0.0\0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
from gensim.utils import simple_preprocess
from gensim.parsing.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import pandas as pd

In [65]:
# Tokenise
#df['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in df['text']] 
# Stem words
porter_stemmer = PorterStemmer()
#df['stemmed_tokens'] = [[porter_stemmer.stem(word) for word in tokens] for tokens in df['tokenized_text'] ]
#Split
df['stemmed_tokens']=df['text']
X_train,X_test,Y_train,Y_test=train_test_split(df['stemmed_tokens'],df['classification'])
print("Value counts for Train sentiments")
print(Y_train.value_counts())
print("Value counts for Test sentiments")
print(Y_test.value_counts())
X_train = X_train.reset_index()
X_test = X_test.reset_index()
Y_train = Y_train.to_frame()
Y_train = Y_train.reset_index()
Y_test = Y_test.to_frame()
Y_test = Y_test.reset_index()

Value counts for Train sentiments
__label__Acceptable      56071
__label__Unacceptable    45596
Name: classification, dtype: int64
Value counts for Test sentiments
__label__Acceptable      18883
__label__Unacceptable    15006
Name: classification, dtype: int64


In [66]:
#Save the files as txt files, required for fasttext
train=pd.concat([Y_train['classification'],X_train['stemmed_tokens']],axis=1)
test=pd.concat([Y_test['classification'],X_test['stemmed_tokens']],axis=1)
train.to_csv('data.train.txt', sep=' ', index=False,encoding="utf-8")
test.to_csv('data.test.txt', sep=' ', index=False,encoding="utf-8")

In [67]:
# fasttext processing
import fasttext
model = fasttext.train_supervised('data.train.txt')
#print(model.words)
print(model.labels)

['__label__Acceptable', '__label__Unacceptable']


In [68]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('data.test.txt'))

N	33889
P@1	0.924
R@1	0.924


In [ ]:
manual_input="I had a lovely time at the beach last week, thanks all for coming :)"
print("The sentence:",manual_input,'. Was deemed ',model.predict(manual_input), 'by the model')

The sentence: I had a lovely time at the beach last week, thanks all for coming :) . Was deemed  (('__label__Acceptable',), array([0.80991381])) by the model
